In [958]:
import pandas as pd
import math
import numpy as np
import requests
import time
from collections import Counter
from sklearn.model_selection import train_test_split

In [959]:
df = pd.read_excel('./data.xlsx', index_col=0)
df.columns

Index(['pair', 'token_name', 'token_address', 'price_b', 'price_s',
       'token_age_b', 'token_age_s', 'transfers', 'buys_m5', 'sells_m5',
       'buys_h1', 'sells_h1', 'buys_h6', 'sells_h6', 'buys_h24', 'sells_h24',
       'volume_m5', 'volume_h1', 'volume_h6', 'volume_h24', 'price_change_m5',
       'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
       'fdv', 'market_cap', 'is_mutable_metadata', 'is_telegram', 'is_twitter',
       'is_website', 'boosts', 'telegram_members', 'is_telegram_error',
       'twitter_days', 'twitter_tweets', 'twitter_followers',
       'twitter_smart_followers', 'is_twitter_error', 'opening_date',
       'closing_date', 'sns_bought', 'sns_sold', 'sns_held_all',
       'sns_sold_some', 'sns_sold_all', 'tt_bought', 'tt_sold', 'PNL',
       'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50', 'status',
       'price_change_check', 'mode'],
      dtype='object')

In [960]:
drop_columns = ['pair', 'token_name',
    'buys_h6', 'sells_h6', 'buys_h24', 'sells_h24',
    'volume_h6', 'volume_h24', 'fdv',
    'price_s', 'token_age_s', 
    'opening_date', 
    #'closing_date',
    'status', 'mode']
df = df.drop(drop_columns, axis=1)
df.head()

,token_address,price_b,token_age_b,transfers,buys_m5,sells_m5,buys_h1,sells_h1,volume_m5,volume_h1,...,sns_sold_all,tt_bought,tt_sold,PNL,PNL_10,PNL_20,PNL_30,PNL_40,PNL_50,price_change_check
id,,,,,,,,,,,,,,,,,,,,,
5807,986h3rNLNCUyHmdCKxPwvjxGFQYF8duekFG4cQRjpump,"0,000155","72,09",48831.0,242,84,49052,1134,"26886,58","438325,29",...,46.0,67.0 53.0 0 98.0 102.0 93.0 53.0 68.0 60.0 61....,2408.0 2277.0 2117.0 1960.0 1911.0 1869.0 1501...,"-20,90",NaN,NaN,NaN,NaN,NaN,"22,820919175911246"
5806,4WFB4bEikmMsVZLsxAsnRiqms4yHSH48WAM29p9Spump,"0,0002683","11,77",6142.0,2399,2725,5242,4981,"759668,81","863418,39",...,55.0,0 0 0 0 779.0 324.0 0 0 0 0 433.0 524.0 0 433....,2718.0 2602.0 2501.0 1402.0 1993.0 1382.0 934....,"-25,79",NaN,NaN,NaN,NaN,NaN,"18,350242611380683"
5805,GVYygChZr2oyVMgSabwhtNhyGjqr5nPtto5aNx94pump,"0,000187","27,39",8311.0,476,389,4467,4101,"198934,89","708714,61",...,60.0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,5037.0 2880.0 1397.0 1319.0 1317.0 1301.0 1296...,"-22,51",NaN,NaN,NaN,NaN,NaN,"9,999999999999986"
5804,8FXyz3V57XC5zaRRMExZb5bZeVdc4CP2zqYtSpuopump,"0,000318","16,35",10069.0,157,63,4438,4237,"25627,89","574889,17",...,59.0,0 0 0 0 777.0 431.0 0 0 215.0 172.0 0 323.0 34...,3971.0 1426.0 1230.0 623.0 1226.0 820.0 286.0 ...,"-24,09",1.0,1.0,1.0,1.0,1.0,"-0,34471952366030295"
5803,FDuKr3pRX5zu6VQQ8wvoUcyBT4jG5AQ4ENuXUEkMpump,"0,0002985","15,87",14701.0,1131,2809,6930,6392,"169434,49","823277,54",...,72.0,635.0 1078.0 0 0 431.0 0 0 0 0 0 64.0 215.0 21...,6207.0 2539.0 1241.0 839.0 1034.0 547.0 496.0 ...,"70,45",1.0,1.0,1.0,1.0,1.0,"15,787432117920874"


In [961]:
#df = df.dropna(subset=['transfers', 'PNL'])
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, 5807 to 5597
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   token_address            177 non-null    object 
 1   price_b                  177 non-null    object 
 2   token_age_b              177 non-null    object 
 3   transfers                177 non-null    float64
 4   buys_m5                  177 non-null    int64  
 5   sells_m5                 177 non-null    int64  
 6   buys_h1                  177 non-null    int64  
 7   sells_h1                 177 non-null    int64  
 8   volume_m5                177 non-null    object 
 9   volume_h1                177 non-null    object 
 10  price_change_m5          177 non-null    object 
 11  price_change_h1          177 non-null    object 
 12  price_change_h6          177 non-null    object 
 13  price_change_h24         177 non-null    object 
 14  liquidity                17

In [962]:
float_types = [
    'token_age_b', 'price_b', 
    'transfers', 'buys_m5', 'sells_m5',
    'buys_h1', 'sells_h1', 
    'volume_m5', 'volume_h1', 'price_change_m5',
    'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
    'market_cap', 'PNL', 'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50',
    'price_change_check'
],

int_types = [
    'is_mutable_metadata', 'is_telegram', 'is_twitter',
    'is_website', 'boosts', 'telegram_members', 'is_telegram_error',
    'twitter_days', 'twitter_tweets', 'twitter_followers',
    'twitter_smart_followers', 'is_twitter_error',
    'sns_held_all', 'sns_sold_some', 'sns_sold_all',
]

for col in float_types:
    df[col] = df[col].replace(",",".", regex=True).astype(float)
    
for col in int_types:
    df[col] = df[col].replace(",",".", regex=True).astype(int)

In [963]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177 entries, 5807 to 5597
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   token_address            177 non-null    object 
 1   price_b                  177 non-null    float64
 2   token_age_b              177 non-null    float64
 3   transfers                177 non-null    float64
 4   buys_m5                  177 non-null    float64
 5   sells_m5                 177 non-null    float64
 6   buys_h1                  177 non-null    float64
 7   sells_h1                 177 non-null    float64
 8   volume_m5                177 non-null    float64
 9   volume_h1                177 non-null    float64
 10  price_change_m5          177 non-null    float64
 11  price_change_h1          177 non-null    float64
 12  price_change_h6          177 non-null    float64
 13  price_change_h24         177 non-null    float64
 14  liquidity                17

In [964]:
#df = df[df['is_telegram_error'] == 0]
#df = df[df['is_twitter_error'] == 0]
#df = df[df['price_change_m5'] > -10]
#df = df[df['price_change_h1'] > 10]
#df = df[(df['transfers'] > 100) & (df['transfers'] < 5000)]

In [965]:
df = df.drop(['is_telegram_error', 'is_twitter_error'], axis=1)

In [966]:
df = df.dropna(subset=['sns_bought', 'sns_sold', 'tt_bought', 'tt_sold'])

In [967]:
df = df[df['sns_bought'] != 0]
df = df[df['sns_sold'] != 0]
df = df[df['tt_bought'] != 0]
df = df[df['tt_sold'] != 0]

In [968]:
def covert_to_list(df, col_name):
    all_lsts = []
    for value_str in df[col_name]:
        lst = [float(x) for x in value_str.split(" ")]
        all_lsts.append(lst)
        
    return all_lsts

In [969]:
df['sns_bought_lst'] = covert_to_list(df, 'sns_bought')
df['sns_sold_lst'] = covert_to_list(df, 'sns_sold')
df['tt_bought_lst'] = covert_to_list(df, 'tt_bought')
df['tt_sold_lst'] = covert_to_list(df, 'tt_sold')

In [970]:
from collections import Counter

def get_most_common(df, col_name, is_zero=True):
    result = []
    for lst in df[col_name]:
        copy_lst = lst.copy()
        if not is_zero:
            if 0.0 in copy_lst:
                copy_lst = list(
                    filter(
                        lambda x: x != 0.0,
                        lst
                    )
                )
        if copy_lst:
            occurence_count = Counter(copy_lst)
            result.append(occurence_count.most_common(1)[0])
        else:
            result.append((0.0, 0.0))
        #print(occurence_count.most_common(1)[0][0])
    return result
    
df['sns_most_common'] = [x[0] for x in get_most_common(df, 'sns_bought_lst')]
df['sns_most_common_except_0'] = [x[0] for x in get_most_common(df, 'sns_bought_lst', False)]

In [971]:
def sum_values(df: pd.DataFrame, col_name: str, num: int) -> pd.DataFrame:
    new_values_lst = []
    for lst in df[col_name]:
        end = len(lst) + 1 if len(lst) < num else num
        result = 0
        for value in lst[:end]:
            result += value
        new_values_lst.append(result)
        
    return new_values_lst

In [972]:
df['tt_f10_bought']= sum_values(df, 'tt_bought_lst', 100)
df['tt_f10_sold'] = sum_values(df, 'tt_sold_lst', 100)
df['sns_f10_bought'] = sum_values(df, 'sns_bought_lst', 100)
df['sns_f10_sold'] = sum_values(df, 'sns_sold_lst', 100)

In [973]:
def get_pnl(df: pd.DataFrame, bought_col: str, sold_col: str) -> list[list]:
    all_bought = []
    for value_str in df[bought_col]:
        bought_lst = [float(x) for x in value_str.split(" ")]
        all_bought.append(bought_lst)
        
    all_sold = []
    for value_str in df[sold_col]:
        sold_lst = [float(x) for x in value_str.split(" ")]
        all_sold.append(sold_lst)
    
    all_pnl = []
    for each_bought, each_sold in zip(all_bought, all_sold):
        each_pnl = [sold - bought if sold else 0 for bought, sold in zip(each_bought, each_sold)]
        all_pnl.append(each_pnl)
    
    return all_pnl

In [974]:
df['sns_pnl'] = get_pnl(df, 'sns_bought', 'sns_sold')
df['tt_pnl'] = get_pnl(df, 'tt_bought', 'tt_sold')

In [975]:
def count_zero(df: pd.DataFrame, col_name: str, amount: int | None=None):
    count_zero = []
    for lst in df[col_name]:
        count_zero.append(lst.count(0))
    
    return count_zero

In [976]:
df['sns_no_bought'] = count_zero(df, 'sns_bought_lst')
df['sns_no_sold'] = count_zero(df, 'sns_sold_lst')
df['tt_no_bought'] = count_zero(df, 'tt_bought_lst')
df['tt_no_sold'] = count_zero(df, 'tt_sold_lst')

In [977]:
def count_pnl_loss(df: pd.DataFrame, col_name: str):
    count_pnl_loss = []
    for pnl_lst in df[col_name]:
        pnl_loss = sum(i < 0 for i in pnl_lst)
        count_pnl_loss.append(pnl_loss)
        
    return count_pnl_loss

In [978]:
df['sns_pnl_loss'] = count_pnl_loss(df, 'sns_pnl')
df['tt_pnl_loss'] = count_pnl_loss(df, 'tt_pnl')

In [979]:
df = df.drop(
    [
        'tt_bought', 'tt_sold', 
        'sns_bought', 'sns_sold', 
        'tt_bought_lst', 'tt_sold_lst', 
        'sns_bought_lst', 'sns_sold_lst', 
        'sns_pnl', 'tt_pnl', 
    ], 
    axis=1
)

In [980]:
def get_pairs_count(token_address):
    time.sleep(1)
    try:
        pairs = requests.get("https://api.dexscreener.com/latest/dex/tokens/" + token_address).json()["pairs"]
        count = len(pairs)
    except:
        count = 0
    return count

In [981]:
df = df.drop(["token_address"], axis=1)

In [982]:
result_df = df.loc[
    (df['sells_m5'] < 400) 
    & (df['sells_h1'] < 1000) 
    #& (df['boosts'] == 500) 
    & (df['sns_held_all'] > 40) 
    & (df['tt_no_sold'] > 10) 
    & (df['buys_h1'] < 10000) 
    & (df['token_age_b'] >= 6)
    & (df['token_age_b'] <= 60)
    & (df['price_b'] >= 0.00001)
    & (df['telegram_members'] >= 1)
    & (df['sns_pnl_loss'] <= 15)
    & (df['tt_pnl_loss'] <= 20)
    & (df['price_change_m5'] > 0)
    & (df['price_change_h1'] > 0)
    #& ((df['price_change_check'] == 0) | (df['price_change_check'] >= 15))
]

profit = result_df.loc[df['PNL_40'] == 1]
loss = result_df.loc[df['PNL_40'] == 0]

print(f"Выигрыш: {profit.shape[0]}")
print(f"Проигрыш: {loss.shape[0]}")

Выигрыш: 13
Проигрыш: 2


In [983]:
df.loc[df["PNL"] >= 60, "profit"] = 1
df.loc[df["PNL"] < 60, "profit"] = 0
df["profit"] = df["profit"].astype(int)

In [984]:
df.to_csv('./clean_data.csv', index=False)

In [985]:
result_df.to_excel('./clean_data.xlsx', index=False)

In [986]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172 entries, 5807 to 5597
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price_b                   172 non-null    float64
 1   token_age_b               172 non-null    float64
 2   transfers                 172 non-null    float64
 3   buys_m5                   172 non-null    float64
 4   sells_m5                  172 non-null    float64
 5   buys_h1                   172 non-null    float64
 6   sells_h1                  172 non-null    float64
 7   volume_m5                 172 non-null    float64
 8   volume_h1                 172 non-null    float64
 9   price_change_m5           172 non-null    float64
 10  price_change_h1           172 non-null    float64
 11  price_change_h6           172 non-null    float64
 12  price_change_h24          172 non-null    float64
 13  liquidity                 172 non-null    float64
 14  market_cap 